yfinance API Documentation: https://ranaroussi.github.io/yfinance/reference/index.html 


In [1]:
import sys 
import os

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from datetime import timezone

import importlib

# Add root path so other subfolders are accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# DB functions for interacting with Azure SQL db
import src.utils.db_azure as db
importlib.reload(db) # Reload it every time the cell is run


# Helper Functions
import src.utils.yfinance_helpers as yfh
importlib.reload(yfh) # Force reload every time the cell is run



<module 'src.utils.yfinance_helpers' from 'c:\\Users\\tomwm\\Documents\\Data_Science\\tm_finance_lab\\src\\utils\\yfinance_helpers.py'>

In [ ]:
# get some stock data
ticker = 'CL=F'

data = yf.download(tickers, interval= '1d', group_by='ticker', start = '2024-01-01', end = '2025-01-01' )

# Clean up multi-index column names to just be the feature names (close, open etc) & make lowercase
data = data.stack(level=0,future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=1)

# remote date as the index
data = data.reset_index()

data.columns = [col.lower() for col in data.columns]

# add current timestamp for future QC
data['updated_at'] = pd.Timestamp.now()




In [ ]:
# get industry& sector for each company

df['industry'] = df['symbol'].apply(yfh.get_industry)
df['sector'] = df['symbol'].apply(yfh.get_sector)


In [6]:
header_dict = {
    "symbol": "symbol",
    "shortName": "asset_name",
    "currency": "currency",
    "exchange": "exchange",
    "marketCap": "market_cap",
    "sharesOutstanding": "shares_outstanding",   
    "sector" : "sector",
    'industry' : 'industry'
}


In [138]:
# keep only needed columns & rename
df_new = df[header_dict.keys()]

df_new.rename(columns=header_dict, inplace=True)

C:\Users\tomwm\AppData\Local\Temp\ipykernel_7996\816988026.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns=header_dict, inplace=True)


In [139]:
# filter out non O&G companies and pref shares
df_new = df_new[df_new['sector']=='Energy']
df_new = df_new[df_new['industry'].str.contains('Oil & Gas',na=False)]
df_new = df_new[~df_new['asset_name'].str.contains('PREF SERIES',na=False)] # get rid of pref share tickers
df_new = df_new[~df_new['asset_name'].str.contains('PREF SER',na=False)] 
df_new = df_new[~df_new['asset_name'].str.contains('PREF',na=False)] 



In [140]:
# add meta data columns
df_new['unit'] = 'share'
df_new['asset_type'] = 'stock'
df_new['updated_at'] = datetime.now()

In [145]:
# Upsert header
azure_engine = get_analytics_azure_engine()
azure_upsert(df_new,azure_engine,'asset_header')